In [22]:
import pandas as pd
import json
import zipfile
import os
import ast 
import pandas as pd
from sklearn.model_selection import train_test_split
import openai

In [2]:
# Set the working directory
workdir = '/Users/andres/Library/CloudStorage/GoogleDrive-alvaro0126@gmail.com/Mi unidad/HKS MPA/2023-II/COMPSCI2015/Project/'
os.chdir(workdir)
# Print the new current working directory to confirm the change
print("New working directory:", os.getcwd())

New working directory: /Users/andres/Library/CloudStorage/GoogleDrive-alvaro0126@gmail.com/Mi unidad/HKS MPA/2023-II/COMPSCI2015/Project


In [3]:
# Load evidences and conditions

with open('/Users/andres/Library/CloudStorage/GoogleDrive-alvaro0126@gmail.com/Mi unidad/HKS MPA/2023-II/COMPSCI2015/Project/DDXPlus_Dataset_English/release_evidences.json') as f:
    evidences = json.load(f)

with open('/Users/andres/Library/CloudStorage/GoogleDrive-alvaro0126@gmail.com/Mi unidad/HKS MPA/2023-II/COMPSCI2015/Project/DDXPlus_Dataset_English/release_conditions.json') as f:
    conditions = json.load(f)

# Read patient data (assuming CSV format)
patients_df = pd.read_csv('/Users/andres/Library/CloudStorage/GoogleDrive-alvaro0126@gmail.com/Mi unidad/HKS MPA/2023-II/COMPSCI2015/Project/DDXPlus_Dataset_English/release_train_patients')

In [4]:
# Function to map evidence ID to its description
def map_evidence_to_description(evidence_id, patient_evidence_value, evidences):
    evidence = evidences.get(evidence_id)
    if not evidence:
        return 'Unknown evidence'

    description = evidence.get('question_en', 'Unknown question')
    
    if evidence['data_type'] in ['C', 'M']:
        value_meaning = evidence.get('value_meaning', {})
        value_description = value_meaning.get(patient_evidence_value, {}).get('en', 'Unknown value')
        description += " (" + value_description + ")"
    
    return description

In [5]:
# Function to map condition ID to its ICD-10 code
def map_condition_to_icd10(condition_id, conditions):
    condition = conditions.get(condition_id)
    if condition:
        return condition.get('icd10-id', 'Unknown')
    return 'Unknown'

In [6]:
# Create DataFrame with required columns
df = pd.DataFrame(columns=['age', 'gender', 'diagnosis', 'ICD 10 code', 'symptoms', 'antecedents'])

In [7]:
print(patients_df.columns)


Index(['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'EVIDENCES',
       'INITIAL_EVIDENCE'],
      dtype='object')


In [12]:
rows = []

for index, row in patients_df.iterrows():
    # Map diagnosis to ICD-10 code
    icd_10_code = map_condition_to_icd10(row['PATHOLOGY'], conditions)

    # Convert symptoms and antecedents to natural language
    symptoms = []
    antecedents = []

    # Convert the 'EVIDENCES' string to a list
    try:
        evidences_list = ast.literal_eval(row['EVIDENCES'])
    except ValueError:
        # Handle cases where 'EVIDENCES' is not a valid list string
        evidences_list = []

    for evidence_str in evidences_list:
        parts = evidence_str.split('_@_')
        evidence_id = parts[0]
        value = parts[1] if len(parts) > 1 else None

        description = map_evidence_to_description(evidence_id, value, evidences)
        if evidences.get(evidence_id, {}).get('is_antecedent', False):
            antecedents.append(description)
        else:
            symptoms.append(description)

    # Append the data to the DataFrame
    rows.append({
        'age': row['AGE'],
        'gender': row['SEX'],
        'diagnosis': row['PATHOLOGY'],
        'ICD 10 code': icd_10_code,
        'symptoms': ' '.join(symptoms),
        'antecedents': ' '.join(antecedents)
    })

df = pd.DataFrame(rows)

print(df.head())

   age gender                diagnosis ICD 10 code  \
0   18      M                     URTI       j06.9   
1   21      M  HIV (initial infection)         B20   
2   19      F                Pneumonia    j17, j18   
3   34      F                     URTI       j06.9   
4   36      M                     URTI       j06.9   

                                            symptoms  \
0  Have you had significantly increased sweating?...   
1  Do you have swollen or painful lymph nodes? Ha...   
2  Do you have pain somewhere, related to your re...   
3  Do you have pain somewhere, related to your re...   
4  Have you had significantly increased sweating?...   

                                         antecedents  
0  Do you live with 4 or more people? Do you smok...  
1  Have you ever had a sexually transmitted infec...  
2  Do you drink alcohol excessively or do you hav...  
3  Do you live with 4 or more people? Have you tr...  
4  Do you attend or work in a daycare? Have you t...  


In [20]:
df.shape

(1025602, 6)

In [16]:
df.columns

Index(['age', 'gender', 'diagnosis', 'ICD 10 code', 'symptoms', 'antecedents'], dtype='object')

In [18]:
len(df['ICD 10 code'].unique())

In [19]:
import random

# Randomly select 10 rows
sampled_rows = df.sample(n=10, random_state=random.seed())

# Extract symptoms and antecedents
sampled_symptoms = sampled_rows['symptoms'].tolist()
sampled_antecedents = sampled_rows['antecedents'].tolist()

# Print the sampled symptoms and antecedents
print("Sampled Symptoms:")
for symptom in sampled_symptoms:
    print(symptom)

print("\nSampled Antecedents:")
for antecedent in sampled_antecedents:
    print(antecedent)


Sampled Symptoms:
Do you have pain that improves when you lean forward? Do you have pain somewhere, related to your reason for consulting? Characterize your pain: (a knife stroke) Characterize your pain: (sharp) Do you feel pain somewhere? (tonsil(R)) Do you feel pain somewhere? (tonsil(L)) Do you feel pain somewhere? (back of the neck) Do you feel pain somewhere? (palace) Do you feel pain somewhere? (pharynx) How intense is the pain? (Unknown value) Does the pain radiate to another location? (nowhere) How precisely is the pain located? (Unknown value) How fast did the pain appear? (Unknown value) Do you have difficulty swallowing, or have a feeling of discomfort/blockage when swallowing? Are you experiencing shortness of breath or difficulty breathing in a significant way? Do you have a fever (either felt or measured with a thermometer)? Have you noticed that you produce more saliva than usual? Have you noticed a high pitched sound when breathing in? Have you noticed that the tone of 

In [29]:
#now, lets transform the questionarie into narratives
#first, we are going to sample our data set, we have 1 million observations
# Stratify the DataFrame based on 'age', 'gender', 'ICD 10 code'

# first, we remove classes with 1 observations
# Combining stratification columns into a single column
df['combined_stratify'] = df['age'].astype(str) + '_' + df['gender'] + '_' + df['ICD 10 code']

# Filter out combinations with fewer than the threshold
min_members_threshold = 2
value_counts = df['combined_stratify'].value_counts()
to_keep = value_counts[value_counts >= min_members_threshold].index
filtered_df = df[df['combined_stratify'].isin(to_keep)]

# Perform stratified sampling on the filtered DataFrame
sampled_df, _ = train_test_split(filtered_df, test_size=0.99, stratify=filtered_df['combined_stratify'], random_state=42)

In [30]:
sampled_df.shape

(10255, 7)

In [ ]:
#API key from OpenAI
openai.api_key = 'xxxxxxxx'

def transform_text_with_gpt3(questionnaire_text):
    try:
        # Framing the prompt with an instruction
        prompt = ("Convert the following medical questionnaire text into a narrative format as if a patient is describing their symptoms:\n" +
                  questionnaire_text)

        response = openai.Completion.create(
            engine="text-davinci-003",  # or another suitable engine
            prompt=prompt,
            max_tokens=150  # Adjust as needed
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print("Error in transforming text:", e)
        return None

# Transform text for each row in the sampled DataFrame
for index, row in sampled_df.iterrows():
    transformed_symptoms = transform_text_with_gpt3(row['symptoms'])

    # Optionally, you can store these back in the DataFrame
    sampled_df.at[index, 'transformed_symptoms'] = transformed_symptoms

In [39]:
sampled_df.head()

,age,gender,diagnosis,ICD 10 code,symptoms,antecedents,combined_stratify,transformed_symptoms
409766,70,M,Acute otitis media,H66.90,"Do you have pain somewhere, related to your re...",Have you traveled out of the country in the la...,70_M_H66.90,I am consulting because of pain. The pain is l...
978500,14,F,Atrial fibrillation,I48.91,Do you feel slightly dizzy or lightheaded? Do ...,Do you have a known issue with one of your hea...,14_F_I48.91,I have been feeling slightly dizzy or lighthea...
687747,27,F,Larygospasm,J38.5,Have you noticed a high pitched sound when bre...,Have you ever been diagnosed with gastroesopha...,27_F_J38.5,"I have noticed that when I take a deep breath,..."
887971,1,F,Larygospasm,J38.5,Have you noticed a high pitched sound when bre...,Do you smoke cigarettes? Have you had a cold i...,1_F_J38.5,"Yes, I have noticed a high pitched sound when ..."
339957,54,M,Anemia,D64.9,"Do you have pain somewhere, related to your re...",Do you have a poor diet? Have you ever had a d...,54_M_D64.9,I am experiencing pain related to the reason I...


In [46]:
csv_file_path = '/Users/andres/Library/CloudStorage/GoogleDrive-alvaro0126@gmail.com/Mi unidad/HKS MPA/2023-II/COMPSCI2015/Project/DDXPlus_Dataset_English/age_sex_symp_data.csv'


In [47]:
sampled_df.to_csv(csv_file_path, index=False)

In [48]:
df = sampled_df[sampled_df['transformed_symptoms'].str.strip() != '']

In [49]:
csv_file_path_2 = '/Users/andres/Library/CloudStorage/GoogleDrive-alvaro0126@gmail.com/Mi unidad/HKS MPA/2023-II/COMPSCI2015/Project/DDXPlus_Dataset_English/age_sex_symp_traindata.csv'
df.to_csv(csv_file_path_2, index=False)